In [ ]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl==0.15.2 triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1" huggingface_hub hf_transfer
    !pip install --no-deps unsloth

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
adapter_path = '/content/drive/My Drive/MOE_DGA/outputs/model'

from unsloth import FastLanguageModel
from transformers import AutoTokenizer

model_path = "outputs"  # Path where the model and tokenizer were saved
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = adapter_path,
    max_seq_length = 2048,
    load_in_4bit = False,  # Optional, depending on your setup
    load_in_8bit = True
)


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.5.7: Fast Llama patching. Transformers: 4.52.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/6.43G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

In [ ]:
from unsloth.chat_templates import get_chat_template

# Ensure the tokenizer uses the same chat template as training
tokenizer = get_chat_template(tokenizer, chat_template="llama-3.1")


In [ ]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

messages = [
    {"role": "user", "content": "Classify the following domain name as dga or normal: visitform.net"},
]

inputs = tokenizer.apply_chat_template(
    messages,
    tokenize = True,
    add_generation_prompt = True, # Must add for generation
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids = inputs, streamer = text_streamer, max_new_tokens = 128,
                   use_cache = True, temperature = 0.1, min_p = 0.1)


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 July 2024

<|eot_id|><|start_header_id|>user<|end_header_id|>

Classify the following domain name as dga or normal: visitform.net<|eot_id|><|start_header_id|>assistant<|end_header_id|>

The traffic is classified as dga.<|eot_id|>


In [ ]:
import requests
import pandas as pd
import numpy as np
import time
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score
import sys
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
from google.colab import drive
import re

families = [
    #'matsnu.gz',
    'suppobox.gz',
    'charbot.gz',
    'gozi.gz'#,
    #'manuelita.gz'#,
    #'rovnix.gz',
    #'deception.gz',
    #'nymaim.gz'
]

runs = 30

for family in families:
    print(family)
    dga = pd.read_csv(f'/content/drive/My Drive/Familias_Test/{family}', chunksize=50)
    legit = pd.read_csv('/content/drive/My Drive/Familias_Test/legit.gz', chunksize=50)
    dfs = []
    for run in range(runs):
        print(f'{run:2}/{runs}', end='\r')
        dfw = pd.concat([dga.get_chunk(), legit.get_chunk()])
        pred = []
        prob = []
        query_time = []
        results = []

        for domain_to_check in dfw.domain.values:
            st = time.time()
            messages = [{"role": "user", "content": "Classify the following domain name as dga or normal:" + domain_to_check}]
            inputs = tokenizer.apply_chat_template(
                messages,
                tokenize=True,
                add_generation_prompt=True,
                return_tensors="pt",
            ).to("cuda")

            # Generar texto completo sin streaming (más fácil para post-procesar)
            output_ids = model.generate(
                input_ids=inputs,
                max_new_tokens=128,
                use_cache=True,
                temperature=1.0,
                do_sample=False,
                top_p=1.0,
            )
            generated_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)

            # Ejemplo de salida: "The traffic is classified as normal."
            # Parseamos la respuesta:
            if "classified as dga" in generated_text.lower():
                label_value = 1
            elif "classified as normal" in generated_text.lower():
                label_value = 0
            else:
                # Caso no esperado, asignar 0 o manejar de otra forma
                label_value = 0

            pred.append(label_value)
            query_time.append(time.time() - st)

        dfw['pred'] = pred
        # dfw['prob'] = prob  # Si tienes probabilidades, descomenta esta línea
        dfw['query_time'] = query_time
        dfw.to_csv(f'/content/drive/My Drive/results/results_Llama3_FineTuning_docker_{family}_{run}.csv.gz', index=False)


suppobox.gz
charbot.gz
gozi.gz


In [ ]:
import requests
import pandas as pd
import numpy as np
import time
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score
import sys
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
from google.colab import drive
import re

families = ['bigviktor.gz',
            'pizd.gz',
            'ngioweb.gz'

           ]

runs = 30

for family in families:
    print(family)
    dga = pd.read_csv(f'/content/drive/My Drive/New_Families/{family}', chunksize=50)
    legit = pd.read_csv('/content/drive/My Drive/Familias_Test/legit.gz', chunksize=50)
    dfs = []

    # Saltar los primeros 30 chunks de legit
    for _ in range(30):
        legit.get_chunk()



    for run in range(runs):
        print(f'{run:2}/{runs}', end='\r')
        dfw = pd.concat([dga.get_chunk(), legit.get_chunk()])
        pred = []
        prob = []
        query_time = []
        results = []

        for domain_to_check in dfw.domain.values:
            st = time.time()
            messages = [{"role": "user", "content": "Classify the following domain name as dga or normal:" + domain_to_check}]
            inputs = tokenizer.apply_chat_template(
                messages,
                tokenize=True,
                add_generation_prompt=True,
                return_tensors="pt",
            ).to("cuda")

            # Generar texto completo sin streaming (más fácil para post-procesar)
            output_ids = model.generate(
                input_ids=inputs,
                max_new_tokens=128,
                use_cache=True,
                temperature=1.0,
                do_sample=False,
                top_p=1.0,
            )
            generated_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)

            # Ejemplo de salida: "The traffic is classified as normal."
            # Parseamos la respuesta:
            if "classified as dga" in generated_text.lower():
                label_value = 1
            elif "classified as normal" in generated_text.lower():
                label_value = 0
            else:
                # Caso no esperado, asignar 0 o manejar de otra forma
                label_value = 0

            pred.append(label_value)
            query_time.append(time.time() - st)

        dfw['pred'] = pred
        # dfw['prob'] = prob  # Si tienes probabilidades, descomenta esta línea
        dfw['query_time'] = query_time
        dfw.to_csv(f'/content/drive/My Drive/results/results_Llama3_FineTuning_docker_{family}_{run}.csv.gz', index=False)


bigviktor.gz
pizd.gz
ngioweb.gz


In [ ]:
"""
families = [
    'matsnu.gz',
    'suppobox.gz',
    'charbot.gz',
    'gozi.gz',
    'manuelita.gz',
    #'rovnix.gz',
    #'deception.gz',
    #'nymaim.gz'
]
"""
def fpr_tpr(y, ypred):
    tn, fp, fn, tp = confusion_matrix(y, ypred).ravel()
    fpr = fp / (fp + tn)  # False Positive Rate
    tpr = tp / (tp + fn)  # True Positive Rate (Recall)
    return fpr, tpr

for family in families:
    acc = []
    pre = []
    rec = []
    f1 = []
    fpr = []
    tpr = []
    qt = []
    qts = []
    for run in range(runs):
        df = pd.read_csv(f'/content/drive/My Drive/results/results_Llama3_FineTuning_docker_{family}_{run}.csv.gz')
        y = (df.label == 'dga').astype(int)
        ypred = df.pred
        acc.append(accuracy_score(y, ypred))
        pre.append(precision_score(y, ypred))
        rec.append(recall_score(y, ypred))
        f1.append(f1_score(y, ypred))
        fpr_value, tpr_value = fpr_tpr(y, ypred)
        fpr.append(fpr_value)
        tpr.append(tpr_value)
        qt.append(df.query_time.mean())
        qts.append(df.query_time.std())
#    print(f'Query time: {np.mean(qt):0.5f}+/-{np.mean(qts)}:0.5f')
    print(f'{family.split(".")[0]:15}: acc:{np.mean(acc):0.2f}±{np.std(acc):.3f} f1:{np.mean(f1):0.2f}±{np.std(f1):.3f} pre:{np.mean(pre):0.2f}±{np.std(pre):.3f} rec:{np.mean(rec):0.2f}±{np.std(rec):.3f}  FPR:{np.mean(fpr):0.2f}±{np.std(fpr):.3f} TPR:{np.mean(tpr):0.2f}±{np.std(tpr):.3f} Query time: {np.mean(qt):0.5f}±{np.mean(qts):0.5f}')


bigviktor      : acc:0.69±0.041 f1:0.72±0.040 pre:0.66±0.037 rec:0.78±0.058  FPR:0.40±0.058 TPR:0.78±0.058 Query time: 0.71162±0.26606
pizd           : acc:0.46±0.042 f1:0.37±0.061 pre:0.44±0.066 rec:0.32±0.059  FPR:0.40±0.059 TPR:0.32±0.059 Query time: 0.66288±0.25254
ngioweb        : acc:0.78±0.032 f1:0.82±0.025 pre:0.71±0.030 rec:0.96±0.029  FPR:0.40±0.056 TPR:0.96±0.029 Query time: 0.70624±0.29302
